In [ ]:
import duckdb
import pandas as pd

# Connect to the database
connection = duckdb.connect("transport_data.db", read_only=False)


query = f""" CREATE TABLE IF NOT EXISTS services AS SELECT * FROM 'C:/Users/ozgur/Downloads/SwissProject/2024-*.csv'"""
connection.execute(query)

query = f""" select * from services limit 10"""
connection.execute(query).df()



query = '''ALTER TABLE services
RENAME COLUMN BETRIEBSTAG TO OPERATING_DAY;

ALTER TABLE services
RENAME COLUMN FAHRT_BEZEICHNER TO TRIP_IDENTIFIER;

ALTER TABLE services
RENAME COLUMN BETREIBER_ID TO OPERATOR_ID;

ALTER TABLE services
RENAME COLUMN BETREIBER_ABK TO OPERATOR_ABK;

ALTER TABLE services
RENAME COLUMN BETREIBER_NAME TO OPERATOR_NAME;

ALTER TABLE services
RENAME COLUMN PRODUKT_ID TO PRODUCT_ID;

ALTER TABLE services
RENAME COLUMN LINIEN_ID TO LINE_ID;

ALTER TABLE services
RENAME COLUMN LINIEN_TEXT TO LINE_TEXT;

ALTER TABLE services
RENAME COLUMN UMLAUF_ID TO CYCLE_ID;

ALTER TABLE services
RENAME COLUMN VERKEHRSMITTEL_TEXT TO TRANSPORT_MODE_TEXT;

ALTER TABLE services
RENAME COLUMN ZUSATZFAHRT_TF TO ADDITIONAL_TRIP_TF;

ALTER TABLE services
RENAME COLUMN FAELLT_AUS_TF TO CANCELLED_TF;

ALTER TABLE services
RENAME COLUMN BPUIC TO BPUIC;

ALTER TABLE services
RENAME COLUMN HALTESTELLEN_NAME TO STOP_NAME;

ALTER TABLE services
RENAME COLUMN ANKUNFTSZEIT TO ARRIVAL_TIME;

ALTER TABLE services
RENAME COLUMN AN_PROGNOSE TO ARRIVAL_PREDICTION;

ALTER TABLE services
RENAME COLUMN AN_PROGNOSE_STATUS TO ARRIVAL_PREDICTION_STATUS;

ALTER TABLE services
RENAME COLUMN ABFAHRTSZEIT TO DEPARTURE_TIME;

ALTER TABLE services
RENAME COLUMN AB_PROGNOSE TO DEPARTURE_PREDICTION;

ALTER TABLE services
RENAME COLUMN AB_PROGNOSE_STATUS TO DEPARTURE_PREDICTION_STATUS;

ALTER TABLE services
RENAME COLUMN DURCHFAHRT_TF TO THROUGH_TF; '''
connection.execute(query).df()




#Monthly row counts
connection = duckdb.connect("transport_data.db", read_only=False)
query = '''select strftime('%m',OPERATING_DAY), count(*) from services group by strftime('%m',OPERATING_DAY) order by strftime('%m',OPERATING_DAY) '''
connection.execute(query).df()

#'REAL' data row rate
query = """
SELECT COUNT(*) as REAL, (SELECT COUNT(*) FROM services) as TOTAL,
    (CAST(COUNT(*) AS FLOAT) / (SELECT COUNT(*) FROM services)) * 100 AS real_ratio
FROM services
WHERE ARRIVAL_PREDICTION_STATUS = 'REAL'
"""
connection.execute(query).df()


#'REAL' data line_id rate
query = """
SELECT COUNT(DISTINCT LINE_ID) as REAL, (SELECT COUNT(DISTINCT LINE_ID) FROM services) as TOTAL,
    (CAST(COUNT(DISTINCT LINE_ID) AS FLOAT) / (SELECT COUNT(DISTINCT LINE_ID) FROM services)) * 100 AS real_ratio
FROM services
WHERE ARRIVAL_PREDICTION_STATUS = 'REAL'
"""
connection.execute(query).df()


#Product Id rates in 'REAL' data line_id based
query = """
SELECT 
    PRODUCT_ID, 
    COUNT(DISTINCT LINE_ID) AS count, 
    ROUND((CAST(COUNT(DISTINCT LINE_ID) AS FLOAT) / (SELECT COUNT(DISTINCT LINE_ID) FROM services WHERE ARRIVAL_PREDICTION_STATUS = 'REAL')) * 100,2) AS percentage
FROM services
WHERE ARRIVAL_PREDICTION_STATUS = 'REAL'
GROUP BY PRODUCT_ID
ORDER BY count DESC
"""
connection.execute(query).df()


#Product Id rates in 'REAL' data distinct row based (where arrival and departure time and prediction is not nul)
query = """SELECT 
    PRODUCT_ID, 
    COUNT(*) AS distinct_line_count, 
    (SELECT COUNT(*) 
     FROM services AS sub 
     WHERE sub.ARRIVAL_PREDICTION_STATUS = 'REAL' 
       AND sub.PRODUCT_ID = services.PRODUCT_ID
    ) AS total_distinct_lines,
    ROUND(
        (CAST(COUNT(*) AS FLOAT) / 
         (SELECT COUNT(*) 
          FROM services AS sub 
          WHERE sub.ARRIVAL_PREDICTION_STATUS = 'REAL' 
            AND sub.PRODUCT_ID = services.PRODUCT_ID
         )
        ) * 100, 2
    ) AS percentage
FROM services
WHERE ARRIVAL_PREDICTION_STATUS = 'REAL'
  AND TRIP_IDENTIFIER NOT IN (
      SELECT DISTINCT TRIP_IDENTIFIER
      FROM services
      WHERE (ARRIVAL_PREDICTION IS NULL AND ARRIVAL_TIME IS NOT NULL)
         OR (DEPARTURE_PREDICTION IS NULL AND DEPARTURE_TIME IS NOT NULL)
  )
GROUP BY PRODUCT_ID
ORDER BY distinct_line_count DESC;"""
connection.execute(query).df()



#clean no arrival prediction data 
query = """SELECT 
    PRODUCT_ID, 
    COUNT(DISTINCT LINE_ID) AS distinct_line_count, 
    (SELECT COUNT(DISTINCT LINE_ID) 
     FROM services AS sub 
     WHERE sub.ARRIVAL_PREDICTION_STATUS = 'REAL' 
       AND sub.PRODUCT_ID = services.PRODUCT_ID
    ) AS total_distinct_lines,
    ROUND(
        (CAST(COUNT(DISTINCT LINE_ID) AS FLOAT) / 
         (SELECT COUNT(DISTINCT LINE_ID) 
          FROM services AS sub 
          WHERE sub.ARRIVAL_PREDICTION_STATUS = 'REAL' 
            AND sub.PRODUCT_ID = services.PRODUCT_ID
         )
        ) * 100, 2
    ) AS percentage
FROM services
WHERE ARRIVAL_PREDICTION_STATUS = 'REAL'
  AND TRIP_IDENTIFIER NOT IN (
      SELECT DISTINCT TRIP_IDENTIFIER
      FROM services
      WHERE (ARRIVAL_PREDICTION IS NULL AND ARRIVAL_TIME IS NOT NULL)
         OR (DEPARTURE_PREDICTION IS NULL AND DEPARTURE_TIME IS NOT NULL)
  )
GROUP BY PRODUCT_ID
ORDER BY distinct_line_count DESC;"""
connection.execute(query).df()



#result----- delayed data
query = f""" 
SELECT epoch((try_strptime(ARRIVAL_TIME, '%d.%m.%Y %H:%M')-ARRIVAL_PREDICTION)) as delay, PRODUCT_ID, strftime('%m',OPERATING_DAY) as Month,BPUIC,STOP_NAME, LINE_ID
FROM services 
WHERE DEPARTURE_PREDICTION_STATUS='REAL' 
AND ARRIVAL_PREDICTION_STATUS='REAL' 
AND TRIP_IDENTIFIER NOT IN (
      SELECT DISTINCT TRIP_IDENTIFIER
      FROM services
      WHERE (ARRIVAL_PREDICTION IS NULL AND ARRIVAL_TIME IS NOT NULL)
         OR (DEPARTURE_PREDICTION IS NULL AND DEPARTURE_TIME IS NOT NULL)
  )
  AND epoch((try_strptime(ARRIVAL_TIME, '%d.%m.%Y %H:%M')-ARRIVAL_PREDICTION))>0
"""
result = connection.sql(query).df()


filtered_result=result[result['delay']<3000]

#top 10 avg most delayed
filtered_result.groupby('STOP_NAME')['delay'].mean().nlargest(10).index


#Delay dist of stops (most delayed top 3)
from matplotlib import pyplot as plt

stop_counts = filtered_result['STOP_NAME'].value_counts()
valid_stops = stop_counts[stop_counts >= 100].index

# Step 3: Find the top 3 stops with the highest delays and at least 10 records
top_3_stops = filtered_result[filtered_result['STOP_NAME'].isin(valid_stops)].groupby('STOP_NAME')['delay'].mean().nlargest(3).index


# Step 3: Create separate histograms for the top 3 stops
for stop in top_3_stops:
    stop_data = filtered_result[filtered_result['STOP_NAME'] == stop]
    
    # Create a new figure for each stop
    plt.figure(figsize=(10, 6))
    
    # Plot the delay distribution for the current stop
    plt.hist(stop_data['delay'], bins=30, density=False, alpha=0.7, color='blue')
    
    # Customize the plot
    plt.title(f'Delay Distribution for {stop}')
    plt.xlabel('Delay (seconds)')
    plt.ylabel('Count')
    
    # Show the plot
    plt.show()
    
    
    
#Delay dist of lines (most delayed top 3)
line_counts = filtered_result['LINE_ID'].value_counts()
valid_lines = line_counts[line_counts >= 100].index

# Step 3: Find the top 3 stops with the highest delays and at least 10 records
top_3_lines = filtered_result[filtered_result['LINE_ID'].isin(valid_lines)].groupby('LINE_ID')['delay'].mean().nlargest(3).index


# Step 3: Create separate histograms for the top 3 stops
for line in top_3_lines:
    line_data = filtered_result[filtered_result['LINE_ID'] == line]
    
    # Create a new figure for each stop
    plt.figure(figsize=(10, 6))
    
    # Plot the delay distribution for the current stop
    plt.hist(line_data['delay'], bins=30, density=False, alpha=0.7, color='blue')
    
    # Customize the plot
    plt.title(f'Delay Distribution for {line}')
    plt.xlabel('Delay (seconds)')
    plt.ylabel('Count')
    
    # Show the plot
    plt.show()

    
    
df_Zug=result[result['PRODUCT_ID']=='Zug']
df_Bus=result[(result['PRODUCT_ID']=='Bus' )| (result['PRODUCT_ID']=='BUS' )]
df_Tram=result[result['PRODUCT_ID']=='Tram']


#Delay Dist of Zug (delayed less than 10mins)
df_Zug=df_Zug[df_Zug['delay']<600]
df_Bus=df_Bus[df_Bus['delay']<600]
df_Tram=df_Tram[df_Tram['delay']<600]
# Quick plotting with pandas
df_Zug['delay'].plot(kind='hist', bins=20, color='blue', density=False, alpha=0.7, figsize=(10, 6))

# Customize the plot
plt.title('Delay Distribution for Zug')
plt.xlabel('Delay (seconds)')
plt.ylabel('Count')

plt.show()

#schema
query = '''select * from INFORMATION_SCHEMA.COLUMNS'''
connection.execute(query).df()